In [ ]:
import pandas as pd
import os
import json

query_dict = {}

with open(os.path.join("..", "data", "tabular_queries_chosen.csv"), "r") as f:
    file_list = pd.read_csv(f)
    
query_ids = file_list["Query ID"]
queries = file_list["Query"]
extract_queries = file_list["Extract Query"]

for query_id, query, extract_query in zip(query_ids, queries, extract_queries):
    with open(os.path.join("..", "data", "collections", "knn", f"{query}.json"), "r") as f:
        collection = json.load(f)
        passages = [collection["hits"][i]["passages"] for i in range(len(collection["hits"]))]

    with open(os.path.join("..", "data", "collections", "knn", "zeta_alpha_prompt", f"{query_id}_{query}.csv"), "r") as f:
        df = pd.read_csv(f)
        # get the gen_answers column
        gen_answers = df["gen_answer"]
        titles = df["Title"]
        urls = df["URL"]
        guids = df["guid"]

        df_answers = []
        df_passages = []
        for answer_list, passage_list in zip(gen_answers, passages):
            final_answers = []
            final_passages = []
            answer_list = eval(answer_list)
            answer_set = set()
            for answer, passage in zip(answer_list, passage_list):
                answer, passage = answer.strip(), passage.strip()
                if answer not in answer_set and answer != "NA":
                    final_answers.append(answer)
                    final_passages.append(passage)
                    answer_set.add(answer)

            if final_answers == []:
                final_answers = ["NA"]
                final_passages = passage_list

            # remove new lines from final passages
            final_passages = [passage.replace("\n", " ") for passage in final_passages]

            df_answers.append(final_answers)
            df_passages.append(final_passages)
        
        df["final_answers"] = df_answers
        df["final_passages"] = df_passages
        df[["guid", "Title", "URL", "final_answers", "final_passages"]].to_csv(os.path.join("..", "data", "collections", "knn", "zeta_alpha_prompt", f"{query_id}.csv"), index=False)
            

In [ ]:
# show all in series
series = query_dict["8d44bd26"]["answers"]
for i in range(len(series)):
    print(series[i])

In [ ]:
for query_id, items in query_dict.items():
    for answer_list, passage_list in zip(items["answers"], items["passages"]):
        answer_list = eval(answer_list)
        # remove duplicates from answers
        final_answers = []
        final_passages = []
        answer_set = set()
        for answer, passage in zip(answer_list, passage_list):
            answer, passage = answer.strip(), passage.strip()
            if answer not in answer_set and answer != "NA":
                final_answers.append(answer)
                final_passages.append(passage)
                answer_set.add(answer)

        if final_answers == []:
            final_answers = ["NA"]
            final_passages = passage_list

        # remove new lines from final passages
        final_passages = [passage.replace("\n", " ") for passage in final_passages]

        query_dict[query_id]["final_answers"] = final_answers
        query_dict[query_id]["final_passages"] = final_passages

    items["final_answers"] = str(items["final_answers"])
    items["final_passages"] = str(items["final_passages"])

    df = pd.DataFrame(items)
    df[["guids", "titles", "urls", "final_answers", "final_passages"]].to_csv(os.path.join("data", "collections", "knn", "zeta_alpha_prompt", f"{query_id}.csv"), index=False)